In [1]:
import numpy as np
from tensorflow import keras
import tensorflow_text as text
import keras_hub
import tensorflow as tf
import string
AUTOTUNE = tf.data.AUTOTUNE

ModuleNotFoundError: No module named 'tensorflow_text'

In [34]:
batch_size = 50  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 1500  # Number of samples to train on.

train_pairs = ("Z:\\official-code\\TRAINING_pairs.txt") # Path to the data txt file on disk.

In [ ]:
def gloss_cleaning(text):
    text.replace("|-->", "")
    text.replace("!", "")
    text.replace("(1h)", "")
    text.replace("(2h)", "")
    text.replace("alt.", "")
    text.replace("+ ", " ")
    return text

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
unsplittables = ["//", "--", "##", "^^", "++", "::", "fs-", "ns-", "IX", "POSS", "SELF"]

with open(train_pairs, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split("\t")
    target_text = gloss_cleaning(target_text)
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

tokenizer = keras_hub.tokenizers.BytePairTokenizer(
    vocabulary=target_texts,
    merges=None,
    sequence_length=None,
    add_prefix_space=False,
    unsplittable_tokens=unsplittables,
    dtype="int32",
)

for char in tokenizer.get_vocabulary():
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

ImportError: BytePairTokenizer requires `tensorflow` and `tensorflow-text` for text processing. Run `pip install tensorflow-text` to install both packages or visit https://www.tensorflow.org/install

If `tensorflow-text` is already installed, try importing it in a clean python session. Your installation may have errors.

KerasHub uses `tf.data` and `tensorflow-text` to preprocess text on all Keras backends. If you are running on Jax or Torch, this installation does not need GPU support.

In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s_model.keras")

Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.7249 - loss: 1.8860 - val_accuracy: 0.8805 - val_loss: 0.6468
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.8617 - loss: 0.7717 - val_accuracy: 0.8805 - val_loss: 0.6178
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.8607 - loss: 0.9351 - val_accuracy: 0.8808 - val_loss: 0.5797
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.8576 - loss: 0.7252 - val_accuracy: 0.8815 - val_loss: 0.5341
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.8611 - loss: 0.6205 - val_accuracy: 0.8812 - val_loss: 0.5270
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.8612 - loss: 0.6288 - val_accuracy: 0.8821 - val_loss: 0.5174
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.8646 - loss: 0.5881 - val_accuracy: 0.8819 - val_loss: 0.5066
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.8528 - loss: 0.6268 - val_accuracy: 0.8820 - val_loss:

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: That first pathway has a visual-tactile orientation. The other path is audiovocal.
Decoded sentence: IX-1p FISE IX-1p FOT IX-1p FINE IX-1p FOT IX-1p FINE IX-1p FOT IX-1p FINE IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p 
-
Input sentence: They talk about the superiority of vision, and the "tyranny of the hegemony of sound".
Decoded sentence: IX-1p FISE IX-1p FOT IX-1p FINE IX-1p FOT IX-1p FINE IX-1p FOT IX-1p FINE IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-1p FOT IX-

KeyboardInterrupt: 